In [1]:
from tmilib import *

In [2]:
from reconstruct_focus_times import ReconstructFocusTimesBaseline
from session_tracker import get_focused_tab

In [54]:

for user in list_users():
  ordered_visits = get_history_ordered_visits_for_user(user)
  if len(ordered_visits) == 0:
    continue
  last_visit = ordered_visits[-1]
  last_visit_time = float(last_visit['visitTime']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)


Eq7EExfolE 2016-03-16 02:39:09.222316
UrZvGUsI8N 2016-03-17 18:53:16.070418
nO1x7w10Yh 2016-03-19 15:48:14.430603
ZDMgTG3hUx 2016-03-19 09:23:49.886797
3H5aGCb1Km 2016-03-19 17:00:52.580188
dwCCD1uofp 2016-03-17 10:28:40.954489
VYfLv7b8hg 2016-03-19 08:42:40.150913
3a3FX1s9S6 2016-03-28 17:45:30.892850
XtIUv7JMHJ 2016-03-16 02:43:02.374703
vbZrd5LD5J 2016-03-16 05:16:58.705542
YLTi7tc27l 2016-03-19 21:06:44.868826
pQiVSc5Ta5 2016-03-20 08:41:39.505303
eS9Wc6ZZ6g 2016-03-17 21:19:22.918957
0Tz22oh1uA 2016-03-28 19:46:03.887097
A7U99GwttI 2016-03-17 21:16:46.360291
7JOMMv7Kqf 2016-03-28 20:49:51.024370
fmFkk3ur1N 2016-03-19 08:45:42.209013
OLAxlg2B3v 2016-03-20 03:40:24.593239
R0JlUdZBdX 2016-03-27 20:02:27.671242
r9pox860Kx 2016-03-16 13:36:44.831024
TfAZTnnzST 2016-03-17 10:58:35.268458
N3buswFgSO 2016-03-16 07:59:38.120257
K6qfMWKpF8 2016-03-16 01:56:49.901559
ZIpD1khEqv 2016-03-18 19:23:48.641418
dIypJPnWfA 2016-03-16 16:11:49.420236
kEVg7v0LcS 2016-03-19 20:18:05.468604
sEFFqggDAp 2

In [4]:
'''
for user in list_users():
  tab_focus_times = get_tab_focus_times_for_user(user)
  if len(tab_focus_times) == 0:
    continue
  last_visit = tab_focus_times[-1]
  last_visit_time = float(last_visit['end']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  if len(tab_focus_times) == 0:\n    continue\n  last_visit = tab_focus_times[-1]\n  last_visit_time = float(last_visit['end']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [5]:
def get_earliest_start_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_earliest_time called with empty list')
  return visit_lengths[0]['start']

def get_last_end_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_last_visit_time called with empty list')
  return visit_lengths[-1]['end']


In [6]:
class UrlAsIndex:
  def __init__(self, offset=0):
    self.index_to_url = []
    self.url_to_index = {}
    for idx in range(offset):
      self.index_to_url.append(None)
  def get_index_for_url(self, url):
    if url in self.url_to_index:
      return self.url_to_index[url]
    index = len(self.index_to_url)
    self.index_to_url.append(url)
    self.url_to_index[index] = url
    return index
  def get_url_for_index(self, index):
    return self.index_to_url[index]

class UrlAtTime:
  def __init__(self, visits, start_time=None, end_time=None):
    if start_time == None:
      start_time = get_earliest_start_time(visits)
    self.offset = start_time # milliseconds
    if end_time == None:
      end_time = get_last_end_time(visits)
    end_time_minus_offset = (end_time - start_time) / 1000.0
    #self.output = [None]*int(round(end_time_minus_offset) + 1)
    self.output = numpy.zeros(int(round(end_time_minus_offset) + 1), dtype=int)
    self.url_as_index = UrlAsIndex(1)
    for visit in visits:
      self.process_visit(visit)
  def process_visit(self, visit):
    url = visit['url']
    url_idx = self.url_as_index.get_index_for_url(url)
    start = visit['start']
    end = visit['end']
    start_idx = int(round((start - self.offset) / 1000.0))
    if start_idx < 0:
      start_idx = 0
      #return
    if start_idx >= len(self.output):
      return
    end_idx = int(round((end - self.offset) / 1000.0))
    if end_idx < 0:
      return
    if end_idx >= len(self.output):
      end_idx = len(self.output) - 1
    for idx in range(start_idx, end_idx+1):
      self.output[idx] = url_idx
  def get_url_for_index(self, index):
    return self.url_as_index.get_url_for_index(index)
  def get_output(self):
    return self.output

def second_to_active_url(visits, start_time, end_time):
  # offset: millseconds
  #start_time = get_earliest_start_time(visits)
  #end_time = get_last_end_time(visits)
  url_at_time = UrlAtTime(visits, start_time, end_time)
  return url_at_time

In [7]:
def ignore_all_before_start_or_after_end(visit_lengths, start_time, end_time):
  output = []
  for x in visit_lengths:
    if x['end'] < start_time:
      continue
    if x['start'] > end_time:
      continue
    #if x['start'] < start_time: # - 1000: # 1 second before or after
    #  continue
    #if x['end'] > end_time: # + 1000:
    #  continue
    output.append(x)
  return output

In [8]:
def evaluation_stats_for_reconstruction(result_reference, result_reconstructed):
  stats = Counter()
  for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output()):
    if ref_answer == 0 and rec_answer == 0:
      stats['both_inactive'] += 1
    elif ref_answer == 0 and rec_answer != 0:
      stats['ref_inactive_but_rec_active'] += 1
    elif ref_answer != 0 and rec_answer == 0:
      stats['ref_active_but_rec_inactive'] += 1
    elif ref_answer != 0 and rec_answer != 0:
      ref_url = result_reference.get_url_for_index(ref_answer)
      rec_url = result_reconstructed.get_url_for_index(rec_answer)
      ref_domain = url_to_domain(ref_url)
      rec_domain = url_to_domain(rec_url)
      if ref_domain == rec_domain:
        if ref_url == rec_url:
          stats['correct_url'] += 1
        else:
          stats['correct_domain'] += 1
      else:
        stats['incorrect_domain'] += 1
  return stats

In [9]:
def get_evaluated_reference_and_reconstructed_focus_times_for_user(user):
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return [],[]
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  return evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times

def get_evaluation_stats_for_user(user):
  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
  #  return {}
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return {}
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  result_reference = second_to_active_url(evaluated_tab_focus_times, ref_start_time, ref_end_time)
  result_reconstructed = second_to_active_url(evaluated_reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  return evaluation_stats_for_reconstruction(result_reference, result_reconstructed)
  #print tab_focus_times[0].keys()
  #print list(reconstructed_tab_focus_times)[0].keys()



In [10]:
class ClosestVisitTimeForUrl:
  def __init__(self, tab_focus_times):
    self.visits = tab_focus_times
    self.url_to_visit_times = {} # url to start of visit
    self.visit_time_to_visit = {}
    for visit in self.visits: # these visits are time-ordered
      start = visit['start']
      url = visit['url']
      if url not in self.url_to_visit_times:
        self.url_to_visit_times[url] = []
      self.url_to_visit_times[url].append(start)
      self.visit_time_to_visit[start] = visit
  def get_visit_from_time(self, target_visit_time):
    self.visit_time_to_visit.get(target_visit_time, None)
  def find_closest_visit(self, url, target_visit_time):
    if url not in self.url_to_visit_times:
      return None
    all_visits = self.url_to_visit_times[url]
    closest_visit = None
    for visit in all_visits:
      if closest_visit == None:
        closest_visit = visit
        continue
      if abs(visit - target_visit_time) < abs(closest_visit - target_visit_time):
        closest_visit = visit
    return closest_visit

In [11]:
def get_span_alignment_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  closest_visit_times_for_reference = ClosestVisitTimeForUrl(evaluated_tab_focus_times)
  closest_visit_times_for_reconstructed = ClosestVisitTimeForUrl(evaluated_reconstructed_tab_focus_times)
  stats = Counter()
  domains_where_reconstructed_has_no_reference = Counter()
  domains_where_reference_has_no_reconstructed = Counter()
  urls_where_reconstructed_has_no_reference = Counter()
  urls_where_reference_has_no_reconstructed = Counter()
  reconstructed_visits_labeled = []
  reference_visits_labeled = []
  for visit in evaluated_reconstructed_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    closest_time = closest_visit_times_for_reference.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if closest_time == None or abs(start_time - closest_time) > 60*1000: # more than 60 seconds off
      stats['reconstructed_has_no_corresponding_reference'] += 1
      domains_where_reconstructed_has_no_reference[domain] += 1
      urls_where_reconstructed_has_no_reference[url] += 1
      visit_labeled['have_reference'] = False
    else:
      stats['reconstructed_has_corresponding_reference'] += 1
      visit_labeled['have_reference'] = True
    reconstructed_visits_labeled.append(visit_labeled)
  for visit in evaluated_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    closest_time = closest_visit_times_for_reconstructed.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if closest_time == None or abs(start_time - closest_time) > 60*1000: # more than 60 seconds off
      stats['reference_has_no_corresponding_reconstructed'] += 1
      domains_where_reference_has_no_reconstructed[domain] += 1
      urls_where_reference_has_no_reconstructed[url] += 1
      visit_labeled['have_reconstructed'] = False
    else:
      stats['reference_has_corresponding_reconstructed'] += 1
      visit_labeled['have_reconstructed'] = True
    reference_visits_labeled.append(visit_labeled)
  return {
    'stats': stats,
    'domains_where_reconstructed_has_no_reference': domains_where_reconstructed_has_no_reference,
    'domains_where_reference_has_no_reconstructed': domains_where_reference_has_no_reconstructed,
    'urls_where_reconstructed_has_no_reference': urls_where_reconstructed_has_no_reference,
    'urls_where_reference_has_no_reconstructed': urls_where_reference_has_no_reconstructed,
    'reconstructed_visits_labeled': reconstructed_visits_labeled,
    'reference_visits_labeled': reference_visits_labeled,
  }

In [12]:
user_to_span_alignment_data = {}

for user in list_users():
  data = get_span_alignment_stats_for_user(user)
  if 'stats' not in data:
    continue
  user_to_span_alignment_data[user] = data


In [13]:
overall_span_alignment_stats = Counter()
for user,data in user_to_span_alignment_data.items():
  stats = data['stats']
  for k,v in stats.items():
    overall_span_alignment_stats[k] += v

print_counter(overall_span_alignment_stats)
# 60% of reference have corresponding reconstructed
# 84% of reconstructed have corresponding reference

reference_has_corresponding_reconstructed 765233
reference_has_no_corresponding_reconstructed 521231
reconstructed_has_corresponding_reference 498695
reconstructed_has_no_corresponding_reference 92425


In [14]:
optimal_reconstruction_stats_spans = Counter()
optimal_reconstruction_stats_times = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reconstructed_visits_labeled']:
    if visit['have_reference']:
      optimal_reconstruction_stats_spans['reconstructed_has_reference'] += 1
    else:
      optimal_reconstruction_stats_spans['reconstructed_has_no_reference'] += 1
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    if visit['have_reconstructed']:
      optimal_reconstruction_stats_spans['reference_has_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_can_be_reconstructed'] += length
    else:
      optimal_reconstruction_stats_spans['reference_has_no_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_cannot_be_reconstructed'] += length

In [15]:
print_counter(optimal_reconstruction_stats_spans)
# reference has reconstructed 60% = 80332.0 / (80332 + 54018)
# reconstructed has reference 84% = 55748.0 / (10948 + 55748)

reference_has_reconstructed 765233
reference_has_no_reconstructed 521231
reconstructed_has_reference 498695
reconstructed_has_no_reference 92425


In [16]:
print_counter(optimal_reconstruction_stats_times)
# seconds of reference that can be reconstructed 57% = 1254885375.005127 / (1254885375.005127 + 947693694.9926758)

seconds_of_reference_that_can_be_reconstructed 10124972703.1
seconds_of_reference_that_cannot_be_reconstructed 10076012470.9


In [17]:
domain_to_seconds_that_cannot_be_reconstructed = Counter()
url_to_seconds_that_cannot_be_reconstructed = Counter()
domain_to_spans_that_cannot_be_reconstructed = Counter()
url_to_spans_that_cannot_be_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    if not visit['have_reconstructed']:
      domain_to_seconds_that_cannot_be_reconstructed[domain] += length
      domain_to_spans_that_cannot_be_reconstructed[domain] += 1
      url_to_seconds_that_cannot_be_reconstructed[url] += length
      url_to_spans_that_cannot_be_reconstructed[url] += 1

In [18]:
print_counter(domain_to_spans_that_cannot_be_reconstructed)

www.mturk.com 145866
newtab 25509
www.facebook.com 24472
mail.google.com 21125
www.youtube.com 20491
www.google.com 15434
www.reddit.com 12285
hitgrabber.net 11727
www.netflix.com 10517
docs.google.com 8659
forums.somethingawful.com 6674
www.twitch.tv 5626
apps.facebook.com 5301
game.soul-arena.com 3946
us-mg5.mail.yahoo.com 3930
us20.chatzy.com 2825
play.pokemonshowdown.com 2722
www.amazon.com 2660
 2537
www.swagbucks.com 2520
calendar.google.com 2507
www.mturkgrind.com 2464
www.tumblr.com 2108
drive.google.com 1514
mturkforum.com 1494
www.mturkcrowd.com 1312
turkernation.com 1245
mail.aol.com 1231
en.wikipedia.org 1117
www.lipstickalley.com 1080
turkopticon.ucsd.edu 1043
klbibkeccnjlkjkiokjodocebajanakg 1024
thewatchseries.to 997
www.ebay.com 982
www.babypips.com 966
web.whatsapp.com 949
allmyvideos.net 941
www.livenewschat.eu 940
www.hbonow.com 919
sbcodez.com 911
forums.animez.to 901
www.linkedin.com 894
boards.4chan.org 873
twitter.com 873
snt151.mail.live.com 864
play.google.com 

In [19]:
print_counter(domain_to_seconds_that_cannot_be_reconstructed)

www.mturk.com 1899414425.98
www.facebook.com 759533278.0
www.youtube.com 636406984.97
mail.google.com 424857526.016
www.reddit.com 406103926.975
www.netflix.com 378153301.995
apps.facebook.com 226973174.003
docs.google.com 186120172.015
forums.somethingawful.com 136441889.996
newtab 113282067.977
www.google.com 110989522.015
us-mg5.mail.yahoo.com 94679892.0027
www.twitch.tv 90361212.9995
hitgrabber.net 75012666.0071
forums.animez.to 67431843.0029
www.amazon.com 60532775.9976
www.tumblr.com 55972974.9963
tweetdeck.twitter.com 55640841.9976
plinga.com 51045024.9983
templates.textbroker.com 50748714.9998
allmyvideos.net 47184916.0012
imgur.com 44447689.9985
play.pokemonshowdown.com 42967681.9956
www.lipstickalley.com 42391648.9978
soundcloud.com 41579309.9998
upenn.co1.qualtrics.com 40398117.9973
games.espn.go.com 36758515.9993
www.hbonow.com 35889428.9966
game.soul-arena.com 35705386.9985
turkernation.com 34364189.0071
projectfreetv.so 32034775.0002
www.livenewschat.eu 31224447.9976
www.

In [20]:
print_counter(url_to_spans_that_cannot_be_reconstructed)

https://www.mturk.com/mturk/findhits?match=true?hit_scraper 28250
https://www.mturk.com/mturk/dashboard 27309
chrome://newtab/ 25504
https://www.facebook.com/ 13002
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 12793
http://hitgrabber.net/ 11727
https://mail.google.com/mail/u/0/#inbox 10079
https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8 7271
http://game.soul-arena.com/ 3946
https://us20.chatzy.com/11920092594925 2825
https://www.mturk.com/HM 2370
https://mail.google.com/mail/u/1/#inbox 2272
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 2091
 1999
https://www.mturk.com/mturk/myhits 1599
https://calendar.google.com/calendar/render??pli=1&pli=1#main_7 1455
https://www.facebook.com/messages/jessica.wood.106 1420
https://www.mturk.com/mturk/findhits?match=true 1254
https://web.whatsapp.com/ 947
https://www.reddit.com/ 913
https://docs.google.com/document/d/1q2s4u1jQ4BUVT16nCr-pkW114mMEDeLiTJztWJxp2Jw/edit 911
http://sbcodez.com/ 911
http://www.swagbu

In [21]:
print_counter(url_to_seconds_that_cannot_be_reconstructed)

https://www.mturk.com/mturk/dashboard 573408452.997
https://www.facebook.com/ 466327673.015
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 286173348.992
https://mail.google.com/mail/u/0/#inbox 148382498.018
chrome://newtab/ 113246845.977
https://www.mturk.com/mturk/findhits?match=true 97738007.0032
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 94225664.0095
https://docs.google.com/spreadsheets/d/1ZBzbhT5itY-L75wfjrNVJQt177kWhXh72bCTqCfMQyE/edit#gid=2096979058 75591704.0
http://hitgrabber.net/ 75012666.0071
https://mail.google.com/mail/#inbox 67010996.9966
https://tweetdeck.twitter.com/# 52280260.9993
https://mail.google.com/mail/u/1/#inbox 52248078.0093
https://templates.textbroker.com/mask/ymask.php 50748714.9998
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 48461632.009
https://www.reddit.com/r/AskReddit/comments/4bpjxo/what_items_or_products_are_found_only_in_rich/ 43674310.9998
https://apps.facebook.com/eontictactoe/?fb_source=bookmark&

In [22]:
overall_domains_where_reconstructed_has_no_reference = Counter()
overall_domains_where_reference_has_no_reconstructed = Counter()
overall_urls_where_reconstructed_has_no_reference = Counter()
overall_urls_where_reference_has_no_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['domains_where_reconstructed_has_no_reference'].items():
    overall_domains_where_reconstructed_has_no_reference[k] += v
  for k,v in data['domains_where_reference_has_no_reconstructed'].items():
    overall_domains_where_reference_has_no_reconstructed[k] += v
  for k,v in data['urls_where_reconstructed_has_no_reference'].items():
    overall_urls_where_reconstructed_has_no_reference[k] += v
  for k,v in data['urls_where_reference_has_no_reconstructed'].items():
    overall_urls_where_reference_has_no_reconstructed[k] += v


In [23]:
print_counter(overall_urls_where_reconstructed_has_no_reference)

https://www.mturk.com/mturk/externalSubmit 16332
https://www.mturk.com/mturk/dashboard 1319
https://www.mturk.com/mturk/previewandaccept?groupId=3SFJXQ9EG58Y3WRLXCW2QCVUYYDXDX 1191
https://www.mturk.com/mturk/previewandaccept?groupId=35DGDVUOHKKVOAMD7VZRLQEQ0J7YPA 1147
https://www.mturk.com/mturk/previewandaccept?groupId=3NYGPW9OUZ9VT0WI3H1AZXNS6BEZP1 1099
https://perk.tv/video/6268/perk-app-trailers 1035
https://www.mturk.com/mturk/checkregistration 950
https://www.facebook.com/ 803
https://www.mturk.com/mturk/myhits 760
https://www.mturk.com/mturk/findhits?match=false 432
https://mail.google.com/mail/u/0/#inbox 382
https://mail.aol.com/38947-111/aol-6/en-us/Lite/MessageActions.aspx 335
https://www.mturk.com/mturk/findhits?match=true 326
https://www.reddit.com/r/HITsWorthTurkingFor/new/ 322
https://www.mturk.com/mturk/endsignin 315
https://www.mturk.com/mturk/viewhits?searchWords=&pageNumber=2&searchSpec=HITGroupSearch%23T%231%2310%23-1%23T%23%21only_available_hits%210%21rO0ABXQABHRyd

In [24]:
print_counter(overall_urls_where_reference_has_no_reconstructed)

https://www.mturk.com/mturk/findhits?match=true?hit_scraper 28250
https://www.mturk.com/mturk/dashboard 27309
chrome://newtab/ 25504
https://www.facebook.com/ 13002
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 12793
http://hitgrabber.net/ 11727
https://mail.google.com/mail/u/0/#inbox 10079
https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8 7271
http://game.soul-arena.com/ 3946
https://us20.chatzy.com/11920092594925 2825
https://www.mturk.com/HM 2370
https://mail.google.com/mail/u/1/#inbox 2272
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 2091
 1999
https://www.mturk.com/mturk/myhits 1599
https://calendar.google.com/calendar/render??pli=1&pli=1#main_7 1455
https://www.facebook.com/messages/jessica.wood.106 1420
https://www.mturk.com/mturk/findhits?match=true 1254
https://web.whatsapp.com/ 947
https://www.reddit.com/ 913
https://docs.google.com/document/d/1q2s4u1jQ4BUVT16nCr-pkW114mMEDeLiTJztWJxp2Jw/edit 911
http://sbcodez.com/ 911
http://www.swagbu

In [25]:
print_counter(overall_domains_where_reconstructed_has_no_reference)

www.mturk.com 39196
www.google.com 4514
www.youtube.com 3399
www.facebook.com 2981
www.reddit.com 2209
i.imgur.com 2016
perk.tv 1894
docs.google.com 1599
mail.google.com 1320
imgur.com 1213
i.ytimg.com 947
mail.aol.com 796
www.amazon.com 730
www.mturkgrind.com 727
mturkforum.com 593
turkernation.com 422
turkopticon.ucsd.edu 363
www.yelp.com 338
www.netflix.com 331
local.cs-int-592.com:5550 327
www.surveymonkey.com 307
www.ebay.com 298
www.bing.com 257
www.earnhoney.com 250
hw.reddit.com 235
www.host2inger.com 225
www.lipstickalley.com 204
thewatchseries.to 199
www.news.com.au 199
surveys.ipsosinteractive.com 177
www.playstationtrophies.org 176
buffer.com 175
my.screenname.aol.com 175
www.yahoo.com 172
www.irazoo.com 163
upload.wikimedia.org 160
accounts.google.com 157
gfycat.com 156
www.instagram.com 153
www.tohosting.tk 149
www.google.co.in 147
en.wikipedia.org 145
workplace.clickworker.com 131
www.lendingclub.com 130
payments.amazon.com 129
mdmarch2016.sawtoothsoftware.com 125
us-mg5

In [26]:
print_counter(overall_domains_where_reference_has_no_reconstructed)

www.mturk.com 121627
www.facebook.com 30306
www.reddit.com 28961
www.youtube.com 26775
v2.decipherinc.com 23656
www.netflix.com 19701
docs.google.com 17475
horriblesubs.info 17356
daemo.herokuapp.com 15792
www.udemy.com 14305
yoga80.com 13550
cmu.qualtrics.com 12173
www.mturkcrowd.com 11747
www.livenewschat.eu 10633
mail.google.com 8221
www.ebay.com 7932
www.mycokerewards.com 5277
www.flexjobs.com 5019
kat.cr 4957
turkitron.com 4809
sites.google.com 4800
turkopticon.ucsd.edu 4792
dragonage.wikia.com 4699
membershipwireless.com 4337
www.hulu.com 4084
www.businessinsider.com 3863
www.countrygeneralstore.com 3809
espn.go.com 3610
www.myfxbook.com 3595
www.classmates.com 3479
funniness.science 3434
mail.aol.com 3206
us.anno-online.com 3189
www.netmeds.com 3064
www.crediblebh.com 3043
howtomakecosmetics.com 2899
www.linkedin.com 2878
uga.view.usg.edu 2876
www.yahoo.com 2823
www.newgrounds.com 2786
www.clickworker.com 2777
boards.na.leagueoflegends.com 2583
vodlocker.com 2489
www.kmtv.com 24

In [27]:
user_to_evaluation_stats = {}

for user in list_users():
  try:
    stats = get_evaluation_stats_for_user(user)
  except:
    continue
  user_to_evaluation_stats[user] = stats


In [28]:
overall_evaluation_stats = Counter()
for user,stats in user_to_evaluation_stats.items():
  for k,v in stats.items():
    overall_evaluation_stats[k] += v

print overall_evaluation_stats

Counter({'both_inactive': 88961569, 'ref_active_but_rec_inactive': 10818171, 'correct_url': 6745088, 'incorrect_domain': 2048236, 'ref_inactive_but_rec_active': 1286180, 'correct_domain': 695229})


In [29]:
'''
def get_users_with_tab_focus_data():
  users_with_tab_focus_data = []

  for user in list_users():
    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
      continue
    tab_focus_times = get_tab_focus_times_for_user(user)
    ordered_visits = get_history_ordered_visits_for_user(user)
    start_time = tab_focus_times[0]['start']
    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]
    #print len(tab_focus_times)
    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:
      users_with_tab_focus_data.append(user)
  return users_with_tab_focus_data
'''
#print len(users_with_tab_focus_data)

"\ndef get_users_with_tab_focus_data():\n  users_with_tab_focus_data = []\n\n  for user in list_users():\n    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):\n      continue\n    tab_focus_times = get_tab_focus_times_for_user(user)\n    ordered_visits = get_history_ordered_visits_for_user(user)\n    start_time = tab_focus_times[0]['start']\n    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]\n    #print len(tab_focus_times)\n    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:\n      users_with_tab_focus_data.append(user)\n  return users_with_tab_focus_data\n"

In [30]:
#user = 'dwCCD1uofp' # doing very poorly with this user
#user = 'UrZvGUsI8N'
#user = users_with_tab_focus_data[0]
#print user
#url_to_visits = get_history_visits_for_user(user)

In [31]:
'''
def compute_tab_focus_times_with_events_for_user(user):
  current_session_tracker = SessionTracker(store_events=True)
  for line in get_log_with_mlog_active_times_for_user(user):
    current_session_tracker.process_input(uncompress_data_subfields(line))
  current_session_tracker.end_input()
  return current_session_tracker.get_output()
'''

'\ndef compute_tab_focus_times_with_events_for_user(user):\n  current_session_tracker = SessionTracker(store_events=True)\n  for line in get_log_with_mlog_active_times_for_user(user):\n    current_session_tracker.process_input(uncompress_data_subfields(line))\n  current_session_tracker.end_input()\n  return current_session_tracker.get_output()\n'

In [32]:
#tab_focus_times_with_events = compute_tab_focus_times_with_events_for_user(user)


In [33]:
'''
for line in get_log_with_mlog_active_times_for_user(user):
  data = uncompress_data_subfields(line)
  url = get_focused_tab(data)
  if url != None:
    print url
    print data
    break
'''

'\nfor line in get_log_with_mlog_active_times_for_user(user):\n  data = uncompress_data_subfields(line)\n  url = get_focused_tab(data)\n  if url != None:\n    print url\n    print data\n    break\n'

In [34]:
#print len(tab_focus_times_with_events)
#tab_focus_times_with_events[0]

In [35]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print len(tab_focus_times)
#print tab_focus_times[0]
#print tab_focus_times

In [36]:
#ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [37]:
'''
ordered_visits_after_start = []
for visit in ordered_visits:
  time = visit['visitTime']
  #if 1458171960840.0 <= time <= 1458172165207.0:
  #  print visit
  if time >= 1458171960840.0:
    ordered_visits_after_start.append(visit)
'''

"\nordered_visits_after_start = []\nfor visit in ordered_visits:\n  time = visit['visitTime']\n  #if 1458171960840.0 <= time <= 1458172165207.0:\n  #  print visit\n  if time >= 1458171960840.0:\n    ordered_visits_after_start.append(visit)\n"

In [38]:
#print len(ordered_visits_after_start)

In [39]:
'''
def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimesBaseline()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)
'''

'\ndef to_visit_lengths(ordered_visits):\n  focus_time_reconstructor = ReconstructFocusTimesBaseline()\n  for visit in ordered_visits:\n    focus_time_reconstructor.process_history_line(visit)\n  return focus_time_reconstructor.get_output()\n\nreconstructed_visit_lengths = to_visit_lengths(ordered_visits)\n'

In [40]:
#print len(reconstructed_visit_lengths)

In [41]:
#print ref_start_time

In [42]:
#print evaluated_tab_focus_times[0]

In [43]:
'''
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break
'''

"\nfor x in reconstructed_visit_lengths:\n  url = x['url']\n  if 'tmi.stanford.edu' in url:\n    print x\n    break\n"

In [44]:
#print len(evaluated_reconstructed_visits)
#print len(evaluated_tab_focus_times)

In [45]:
#def 

In [46]:
#print evaluated_reconstructed_visits[0]
#print evaluated_tab_focus_times[2]
#print evaluated_reconstructed_visits[-1]
#print evaluated_tab_focus_times[-1]

In [47]:
#print len(result_reference.get_output())
#print len(result_reconstructed.get_output())

In [48]:
'''
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:
  ref_domain = 'None'
  rec_domain = 'None'
  if ref_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    ref_domain = url_to_domain(ref_url)
  if rec_answer != 0:
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    rec_domain = url_to_domain(rec_url)
  print ref_domain, rec_domain
'''

"\nfor ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:\n  ref_domain = 'None'\n  rec_domain = 'None'\n  if ref_answer != 0:\n    ref_url = result_reference.get_url_for_index(ref_answer)\n    ref_domain = url_to_domain(ref_url)\n  if rec_answer != 0:\n    rec_url = result_reconstructed.get_url_for_index(rec_answer)\n    rec_domain = url_to_domain(rec_url)\n  print ref_domain, rec_domain\n"

In [49]:
#for x in evaluated_reconstructed_visits:
#  print x

In [50]:
#for x in evaluated_tab_focus_times:
#  print x

In [51]:
#for x in reconstructed_visit_lengths:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [52]:
#for x in tab_focus_times:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [53]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"